<h5 style="color:green"> All Models' features need to be 2D Arrays even if there is one feature </h3>
<ul>
<li> No documnetation
<li> Better OOP Design and Redundancy Omitting
<li> To be implemented :
    <ul>
        <li> Gaussian Naive Bayes
        <li> Logistic Regression
        <li> Support Vector Machine
        <li> Gradient Boosting
        <li> DBSCAN and HDBSCAN
        <li> K-Means
        <li> PCA - UMAP
        <li> Reinforcement Learning
        <li> AlphaZero
        <li> Factorization Methods
        <li> Convolutional Neural Networks
        <li> RNN + LSTM
        <li> Transformers
    </ul>
<li> Needs Better Implementations :
    <ul>
        <li> Faster BallTree / KDTree Algorithms for KNN
    </ul>
</ul>

In [1]:
import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier as SklearnGBC
from sklearn.datasets import make_classification
from sklearn.ensemble import AdaBoostClassifier as SklearnAdaBoost
from sklearn.impute import SimpleImputer as SklearnSimpleImputer
from sklearn.impute import KNNImputer as SklearnKNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer as SklearnIterativeImputer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score as sk_cross_val_score
from sklearn.model_selection import GridSearchCV as SklearnGridSearchCV
from sklearn.model_selection import train_test_split as sklearn_train_test_split
from sklearn.preprocessing import OneHotEncoder as SklearnOneHotEncoder
from sklearn.preprocessing import LabelEncoder as SklearnLabelEncoder
from sklearn.preprocessing import OrdinalEncoder as SklearnOrdinalEncoder
from sklearn.feature_selection import SelectKBest as SklearnSelectKBest, f_classif
from sklearn.feature_selection import SelectFromModel as SklearnSelectFromModel
from sklearn.feature_selection import RFE as SklearnRFE

import decision_tree
import random_forest
import linear_regression
import naive_bayes
import nearest_neighbors
from gradient_boosting import GradientBoostingClassifier
from adaboost import AdaBoostClassifier
from imputation import SimpleImputer
from imputation import KNNImputer
from imputation import IterativeImputer
from kfold import KFold
from grid_search import GridSearchCV
from splitting import train_test_split
from encoding import OneHotEncoder
from encoding import LabelEncoder
from encoding import FrequencyEncoder
from encoding import OrdinalEncoder
from feature_selection import RFE, SelectFromModel, SelectKBest

from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import datasets
import pickle

import time
import matplotlib.pyplot as plt

In [2]:
def load_dataset(filename):
    with open(filename, "rb") as f:
        return pickle.load(f)

In [7]:
def compareModels(models, X, y , metric):
    for i, model in enumerate(models):
        print(f"Model {i}")
        start = time.perf_counter()
        model.fit(X[0], y[0])
        end = time.perf_counter()
        print(f"Time to fit : {end - start} s")
        print(f"Performance : {metric(model.predict(X[1]), y[1])}")


In [4]:
df = load_dataset("citiesSmall.pkl")
X = df['X']
y = df['y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=19)
X_train.shape

(300, 2)

In [5]:
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans as SklearnKMeans
from kmeans import KMeans

# Generate synthetic data
X, _ = make_blobs(n_samples=100, centers=3, n_features=2, random_state=42)

# Instantiate and fit KMeans implementation
kmeans_model = KMeans(num_clusters=3, max_iterations=100)
kmeans_model.fit(X)

# Instantiate and fit sklearn's KMeans
sklearn_kmeans = SklearnKMeans(n_clusters=3, max_iter=100)
sklearn_kmeans.fit(X)

# Compare cluster centers
print("Your KMeans Cluster Centers:")
print(kmeans_model.centroids)
print("\nSklearn's KMeans Cluster Centers:")
print(sklearn_kmeans.cluster_centers_)

# Compare labels
your_labels = kmeans_model.transform(X)
sklearn_labels = sklearn_kmeans.predict(X)
print("\nYour KMeans Labels:")
print(your_labels)
print("\nSklearn's KMeans Labels:")
print(sklearn_labels)

# Check if labels match
matching_labels = np.array_equal(your_labels, sklearn_labels)
print("\nDo the labels match between your implementation and sklearn's KMeans?", matching_labels)


Your KMeans Cluster Centers:
[[-2.66780392  8.93576069]
 [ 4.49951001  1.93892013]
 [-6.95170962 -6.67621669]]

Sklearn's KMeans Cluster Centers:
[[-6.95170962 -6.67621669]
 [ 4.49951001  1.93892013]
 [-2.66780392  8.93576069]]

Your KMeans Labels:
[2. 1. 0. 1. 2. 1. 0. 1. 1. 0. 0. 2. 2. 0. 0. 2. 2. 0. 2. 2. 0. 2. 2. 0.
 0. 0. 1. 2. 2. 2. 2. 1. 1. 2. 0. 0. 0. 0. 1. 1. 2. 0. 1. 0. 0. 1. 2. 2.
 2. 1. 1. 1. 0. 2. 2. 2. 0. 0. 1. 0. 2. 1. 2. 1. 2. 2. 1. 2. 1. 1. 1. 2.
 2. 0. 1. 2. 1. 2. 1. 1. 0. 1. 0. 2. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 0.
 0. 1. 2. 0.]

Sklearn's KMeans Labels:
[0 1 2 1 0 1 2 1 1 2 2 0 0 2 2 0 0 2 0 0 2 0 0 2 2 2 1 0 0 0 0 1 1 0 2 2 2
 2 1 1 0 2 1 2 2 1 0 0 0 1 1 1 2 0 0 0 2 2 1 2 0 1 0 1 0 0 1 0 1 1 1 0 0 2
 1 0 1 0 1 1 2 1 2 0 2 2 2 1 2 1 1 1 2 1 2 2 2 1 0 2]

Do the labels match between your implementation and sklearn's KMeans? False


c:\Users\Apo\anaconda3\envs\MLearning\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [7]:
from sklearn.decomposition import PCA as SKPCA
from pca import PCA  # Import your PCA module here

# Generate some random data
np.random.seed(0)
X = np.random.rand(100, 5)

# Number of principal components
n_components = 3    

# Instantiate your PCA class
pca_transformer = PCA(n_components=n_components)

# Fit and transform data using your PCA
X_transformed_yours = pca_transformer.fit_transform(X)

# Instantiate scikit-learn's PCA
sklearn_pca = SKPCA(n_components=n_components)

# Fit and transform data using scikit-learn's PCA
X_transformed_sklearn = sklearn_pca.fit_transform(X)

# Compare the transformed data
print("Transformed data using your PCA:")
print(X_transformed_yours)

print("\nTransformed data using scikit-learn's PCA:")
print(X_transformed_sklearn)

# Compare the components
print("\nComponents obtained from your PCA:")
print(pca_transformer.components)

print("\nComponents obtained from scikit-learn's PCA:")
print(sklearn_pca.components_)

# Check if the components are almost equal
print("\nAre the components almost equal?")
print(np.allclose(pca_transformer.components, sklearn_pca.components_))

# Check if the transformed data is almost equal
print("\nIs the transformed data almost equal?")
print(np.allclose(X_transformed_yours, X_transformed_sklearn))

Transformed data using your PCA:
[[ 0.12893433  0.00552079 -0.09370797]
 [ 0.41533495 -0.00333602 -0.41972944]
 [ 0.12969153  0.16412773 -0.62603473]
 [ 0.24650397 -0.14045993  0.23768159]
 [ 0.16207752  0.14991566 -0.56898737]
 [ 0.13934388 -0.43966464 -0.29926161]
 [-0.20657412  0.29988263 -0.22675732]
 [ 0.45644469  0.14116105 -0.08405428]
 [-0.08338814 -0.44010773  0.28364689]
 [-0.40669395 -0.16223567 -0.08345219]
 [-0.07822468 -0.54689644 -0.24763786]
 [-0.32185981  0.27818679  0.05877127]
 [-0.46903245 -0.33884216 -0.03807255]
 [-0.18252448  0.60959633 -0.21918938]
 [ 0.56450686 -0.45829395 -0.20635187]
 [-0.69934854  0.14949441  0.05822362]
 [-0.17969097  0.29630788  0.16000299]
 [-0.01618035  0.20430776  0.5082468 ]
 [-0.19010877  0.4347466  -0.03760165]
 [-0.40382975  0.63300159 -0.21056112]
 [ 0.2146383   0.01376866 -0.52437981]
 [ 0.18764827 -0.34837748  0.39123896]
 [ 0.27028174 -0.18019686  0.33089454]
 [ 0.43661432  0.31189231  0.09986693]
 [ 0.35214634 -0.25370294 -0.32

In [4]:
from svm import SVM
from sklearn.svm import SVC as SKSVC

# Generate random data for binary classification
X, y = make_classification(n_samples=5000, n_features=5, n_classes=2, random_state=42)

# Instantiate your SVM class
svm = SVM()

# Instantiate scikit-learn's SVM
sklearn_svm = SKSVC()

# Compare the models
compareModels([svm, sklearn_svm], [X, X], [y, y], accuracy_score)

Model 0
Time to fit : 341.07965860003605 s
Performance : 0.8787
Model 1
Time to fit : 1.5841489000013098 s
Performance : 0.9384


In [11]:
from sklearn.preprocessing import StandardScaler
from logistic_regression import LogisticRegression  # Import your logistic regression module
from sklearn.linear_model import LogisticRegression as SKlogistic

# Generate synthetic data
X, y = make_classification(n_samples=5000, n_features=5, n_classes=2, random_state=42)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


compareModels([LogisticRegression, SKlogistic], [X_train, X_test], [y_train, y_test], accuracy_score)

[0 1 0 ... 1 1 1]
Model 0


TypeError: LogisticRegression.fit() missing 1 required positional argument: 'target_vector'

In [5]:
X, y = datasets.make_regression(n_samples=2000, n_features=5, noise=1, bias=19, random_state=4)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=19)

for i in range(3, 24, 3):
    print("==========================================")
    models = []
    models.append(DecisionTreeRegressor(max_depth=i))
    models.append(decision_tree.DecisionTreeRegressor(max_depth=i))

    compareModels(models, (X_train, X_test), (y_train, y_test), mean_squared_error)
    

Model 0
Time to fit : 0.0035049999714829028 s
Performance : 4901.238641239901
Model 1
Time to fit : 1.832673099997919 s
Performance : 4901.238641239901
Model 0
Time to fit : 0.0055127000086940825 s
Performance : 2538.0615336933442
Model 1
Time to fit : 3.2793744999798946 s
Performance : 2538.0615336933442
Model 0
Time to fit : 0.007210300012957305 s
Performance : 1698.570573640517
Model 1
Time to fit : 4.734253100003116 s
Performance : 1570.7135079619468
Model 0
Time to fit : 0.008474300033412874 s
Performance : 1546.8768444933382
Model 1
Time to fit : 5.633250100014266 s
Performance : 1401.4342614015302
Model 0
Time to fit : 0.010433799994643778 s
Performance : 1497.2174102971874
Model 1
Time to fit : 6.118457000004128 s
Performance : 1391.1640006695484
Model 0
Time to fit : 0.009108399972319603 s
Performance : 1542.6152504494326
Model 1
Time to fit : 6.516799700038973 s
Performance : 1391.7274833228405
Model 0
Time to fit : 0.010205099999438971 s
Performance : 1510.2693828479696
Mode

In [5]:
for i in range(1, 24, 3):
    print("==========================================")
    models = []
    models.append(DecisionTreeClassifier(criterion='gini', max_depth=i))
    models.append(decision_tree.DecisionTreeClassifier(criterion='gini', max_depth=i))

    compareModels(models, (X_train, X_test), (y_train, y_test), accuracy_score)
    

Model 0
Time to fit : 0.0012295999986235984 s
Performance : 0.67
Model 1
Time to fit : 0.07478760000230977 s
Performance : 0.67
Model 0
Time to fit : 0.0005858000004081987 s
Performance : 0.75
Model 1
Time to fit : 0.0526785000001837 s
Performance : 0.75
Model 0
Time to fit : 0.0003744000023289118 s
Performance : 0.8
Model 1
Time to fit : 0.06219740000233287 s
Performance : 0.79
Model 0
Time to fit : 0.00041280000004917383 s
Performance : 0.81
Model 1
Time to fit : 0.07568480000190903 s
Performance : 0.8
Model 0
Time to fit : 0.0006623000008403324 s
Performance : 0.85
Model 1
Time to fit : 0.08843170000181999 s
Performance : 0.85
Model 0
Time to fit : 0.0007795999990776181 s
Performance : 0.83
Model 1
Time to fit : 0.09760359999927459 s
Performance : 0.82
Model 0
Time to fit : 0.0007207000016933307 s
Performance : 0.85
Model 1
Time to fit : 0.10250649999943562 s
Performance : 0.85
Model 0
Time to fit : 0.0006769999999960419 s
Performance : 0.86
Model 1
Time to fit : 0.1540285999981279 

In [ ]:
X, y = datasets.make_classification(n_samples=1000, n_features=5, n_informative=3, n_redundant=2, n_classes=3, random_state=19)

X_categorical = pd.DataFrame()
for i in range(X.shape[1]):
    categorical_feature = pd.cut(X[:, i], bins=5, labels=[j for j in range(5)])
    X_categorical[f'feature_{i}'] = categorical_feature

X = X_categorical.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=19)

for i in range(1, 24, 3):
    print(f"Depth {i} ==========================================")
    models = []
    models.append(RandomForestClassifier(criterion='gini', max_depth=i))
    models.append(random_forest.RandomForestClassifier(criterion='gini', max_depth=i))

    compareModels(models, (X_train, X_test), (y_train, y_test), accuracy_score)

Depth 1 ==========================================
Model 0
Time to fit : 0.060916399999769055 s
Performance : 0.7
Model 1
Time to fit : 0.09508350000032806 s


In [ ]:
X, y = datasets.make_regression(n_samples=10000, n_features=10, noise=1, bias=19, random_state=4)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=19)

models = [LinearRegression(), linear_regression.LinearRegression()]
compareModels(models, (X_train, X_test), (y_train, y_test), mean_squared_error)

# fig = plt.figure(figsize=(10, 8))
# plt.scatter(X, y, marker='o')
# plt.plot(X, model.predict(X), marker = 'o')
# plt.show()

In [ ]:
X, y = datasets.make_classification(n_samples=18000, n_features=5, n_informative=5, n_redundant=0, n_classes=2, random_state=19)

X_categorical = pd.DataFrame()
for i in range(X.shape[1]):
    categorical_feature = pd.cut(X[:, i], bins=5, labels=[j for j in range(5)])
    X_categorical[f'feature_{i}'] = categorical_feature

X = X_categorical.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=19)

models = [MultinomialNB(), naive_bayes.NaiveBayes()]
compareModels(models, (X_train, X_test), (y_train, y_test), accuracy_score)

In [ ]:
X, y = datasets.make_classification(n_samples=1000, n_features=5, n_informative=3, n_redundant=2, n_classes=3, random_state=19)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=19)

for k in range(5, 101):
    print(f"\n{k} neighbors ============================\n")
    models = [KNeighborsClassifier(n_neighbors=k), nearest_neighbors.KNN(k)]
    compareModels(models, (X_train, X_test), (y_train, y_test), accuracy_score)

In [ ]:
from deep_learning import *
from sklearn.preprocessing import StandardScaler

iris = datasets.load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X.shape)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

mlp = Module()

# Add layers to your MLP model
mlp.add_layer(Layer(input_dim=X_train.shape[1], output_dim=64))  # Adjust the dimensions as needed
mlp.add_layer(Sigmoid(64, 64))  # Add activation layers and adjust dimensions as needed
mlp.add_layer(Layer(64, 1))

# Fit the model to the training data
mlp.fit(X_train, y_train)

# Test the model on the testing data
predictions = mlp.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")

In [5]:
# Load dataset
iris = datasets.load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# Fit custom Gaussian Naive Bayes
gnb_custom = naive_bayes.GaussianNaiveBayes()
gnb_custom.fit(X_train, y_train)
y_pred_custom = gnb_custom.predict(X_test)

# Fit sklearn Gaussian Naive Bayes for comparison
gnb_sklearn = GaussianNB()
gnb_sklearn.fit(X_train, y_train)
y_pred_sklearn = gnb_sklearn.predict(X_test)

# Compare results
custom_accuracy = np.mean(y_pred_custom == y_test)
sklearn_accuracy = np.mean(y_pred_sklearn == y_test)

print("Custom Gaussian Naive Bayes Accuracy:", custom_accuracy)
print("Sklearn Gaussian Naive Bayes Accuracy:", sklearn_accuracy)

Custom Gaussian Naive Bayes Accuracy: 0.9666666666666667
Sklearn Gaussian Naive Bayes Accuracy: 0.9666666666666667


In [6]:
# Load dataset
iris = datasets.load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# Fit custom Gradient Boosting Classifier
gb_custom = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3)
gb_custom.fit(X_train, y_train)
y_pred_custom = gb_custom.predict(X_test)

# Fit sklearn Gradient Boosting Classifier for comparison
gb_sklearn = SklearnGBC(n_estimators=100, learning_rate=0.1, max_depth=3)
gb_sklearn.fit(X_train, y_train)
y_pred_sklearn = gb_sklearn.predict(X_test)

# Compare results
custom_accuracy = accuracy_score(y_test, y_pred_custom)
sklearn_accuracy = accuracy_score(y_test, y_pred_sklearn)

print("Custom Gradient Boosting Accuracy:", custom_accuracy)
print("Sklearn Gradient Boosting Accuracy:", sklearn_accuracy)

# Use the exi

c:\Users\ABDERRAHMANE\Desktop\third year\AI\mini-sickitlearn\ZAI-Scikit\gradient_boosting.py:73: RuntimeWarning: invalid value encountered in log
  self.init_pred = np.log(np.sum(y) / (len(y) - np.sum(y)))


Custom Gradient Boosting Accuracy: 0.43333333333333335
Sklearn Gradient Boosting Accuracy: 0.9666666666666667


In [7]:
X, y = make_classification(n_samples=1000, n_features=20, n_informative=2, n_redundant=0, n_clusters_per_class=1, random_state=42)
y = np.where(y == 0, -1, 1)  # AdaBoost in sklearn handles labels {0, 1}, converting to {-1, 1} for custom model

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the custom AdaBoost Classifier
custom_model = AdaBoostClassifier(n_clf=30)
custom_model.fit(X_train, y_train)
custom_preds = custom_model.predict(X_test)
custom_accuracy = accuracy_score(y_test, custom_preds)

# Initialize the sklearn AdaBoost Classifier
sklearn_model = SklearnAdaBoost(n_estimators=30, random_state=42, algorithm='SAMME')  # SAMME is the discrete boost algorithm similar to used in the custom implementation
sklearn_model.fit(X_train, y_train)
sklearn_preds = sklearn_model.predict(X_test)
sklearn_accuracy = accuracy_score(y_test, sklearn_preds)

# Output the results
print("Custom AdaBoost Accuracy:", custom_accuracy)
print("Sklearn AdaBoost Accuracy:", sklearn_accuracy)

# For probability comparison, if implemented
custom_prob = custom_model.predict_proba(X_test[0].reshape(1, -1))
sklearn_prob = sklearn_model.predict_proba(X_test[0].reshape(1, -1))
print("Custom AdaBoost Sample Probabilities:", custom_prob)
print("Sklearn AdaBoost Sample Probabilities:", sklearn_prob)

Custom AdaBoost Accuracy: 0.915
Sklearn AdaBoost Accuracy: 0.925
Custom AdaBoost Sample Probabilities: [[0.84746598 0.15253402]]
Sklearn AdaBoost Sample Probabilities: [[0.56511477 0.43488523]]


In [8]:
# Generating synthetic data
np.random.seed(0)
X, _ = make_classification(n_samples=100, n_features=5, random_state=42)
X = X.astype(float)
X[::4, 1] = np.nan  # Introduce missing values in the second column
X[::3, 3] = np.nan  # Introduce missing values in the fourth column

# Split the dataset to demonstrate usage
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

# Define custom and sklearn imputers
imputers = {
    "mean": (SimpleImputer(strategy="mean"), SklearnSimpleImputer(strategy="mean")),
    "median": (SimpleImputer(strategy="median"), SklearnSimpleImputer(strategy="median")),
    "mode": (SimpleImputer(strategy="mode"), SklearnSimpleImputer(strategy="most_frequent")),
    "knn": (KNNImputer(), SklearnKNNImputer()),
    # "iterative": (IterativeImputer(), SklearnIterativeImputer())
}

# Test each imputer
for name, (custom_imputer, sklearn_imputer) in imputers.items():
    custom_filled = custom_imputer.fit_transform(X_train)
    sklearn_filled = sklearn_imputer.fit_transform(X_train)

    # Compare using MSE to evaluate how close the custom implementation is to sklearn's
    mse = mean_squared_error(custom_filled, sklearn_filled)
    print(f"{name.capitalize()} Imputer MSE between custom and sklearn: {mse:.4f}")

Mean Imputer MSE between custom and sklearn: 0.0000
Median Imputer MSE between custom and sklearn: 0.0000
Mode Imputer MSE between custom and sklearn: 0.0000
Knn Imputer MSE between custom and sklearn: 0.0745


In [9]:
def cross_val_score(model, X, y, scoring, cv=None):
    if cv is None:
        cv = KFold()

    scores = []
    for train_idx, test_idx in cv.split(X):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        
        score = scoring(y_test, predictions)
        scores.append(score)
    
    return np.array(scores)

In [10]:
# Load Iris dataset
iris = datasets.load_iris()
X, y = iris.data, iris.target

# Define the scoring function
def accuracy_scorer(y_true, y_pred):
    return np.mean(y_true == y_pred)    

# Use custom KFold and cross-validation
custom_cv = KFold(n_splits=3, shuffle=True, random_state=42)
model = DecisionTreeClassifier()
scores = cross_val_score(model, X, y, scoring=accuracy_scorer, cv=custom_cv)
print("Custom Implementation Scores:", scores)

# Compare with scikit-learn's implementation
sk_scores = sk_cross_val_score(DecisionTreeClassifier(), X, y, cv=3)
print("Scikit-Learn Implementation Scores:", sk_scores)

Custom Implementation Scores: [0.94 0.96 0.96]
Scikit-Learn Implementation Scores: [0.98 0.92 1.  ]


In [11]:
# Load Iris dataset
data = datasets.load_iris()
X, y = data.data, data.target

# Define cross-validation setup
kfolds = KFold(n_splits=3, shuffle=True, random_state=42)

# Parameter grid
param_grid = {
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 3, 4]
}

# Custom GridSearchCV implementation
custom_grid_search = GridSearchCV(
    estimator=DecisionTreeClassifier(),
    param_grid=param_grid,
    scoring=accuracy_scorer,
    cv=kfolds
)

# Fit custom Grid Search
custom_grid_search.fit(X, y)
print("Custom GridSearchCV - Best Parameters:", custom_grid_search.best_params())
print("Custom GridSearchCV - Best Score:", custom_grid_search.best_score())

# Scikit-learn GridSearchCV
sklearn_grid_search = SklearnGridSearchCV(
    estimator=DecisionTreeClassifier(),
    param_grid=param_grid,
    scoring='accuracy',
    cv=3  # Number of folds, directly passed as an int to use default KFold in sklearn
)

# Fit scikit-learn Grid Search
sklearn_grid_search.fit(X, y)
print("Sklearn GridSearchCV - Best Parameters:", sklearn_grid_search.best_params_)
print("Sklearn GridSearchCV - Best Score:", sklearn_grid_search.best_score_)


Custom GridSearchCV - Best Parameters: {'max_depth': None, 'min_samples_split': 2}
Custom GridSearchCV - Best Score: 0.9533333333333333
Sklearn GridSearchCV - Best Parameters: {'max_depth': None, 'min_samples_split': 4}
Sklearn GridSearchCV - Best Score: 0.9733333333333333


In [12]:
# Load Iris dataset
data = datasets.load_iris()
X, y = data.data, data.target

# Using the custom train_test_split
X_train_custom, X_test_custom, y_train_custom, y_test_custom = train_test_split(X, y, test_size=0.2, random_state=42)

# Using scikit-learn's train_test_split
X_train_sklearn, X_test_sklearn, y_train_sklearn, y_test_sklearn = sklearn_train_test_split(X, y, test_size=0.2, random_state=42)

# Compare the outputs
print("Custom Implementation:")
print("Train shapes (X, y):", X_train_custom.shape, y_train_custom.shape)
print("Test shapes (X, y):", X_test_custom.shape, y_test_custom.shape)

print("\nSklearn Implementation:")
print("Train shapes (X, y):", X_train_sklearn.shape, y_train_sklearn.shape)
print("Test shapes (X, y):", X_test_sklearn.shape, y_test_sklearn.shape)


Custom Implementation:
Train shapes (X, y): (120, 4) (120,)
Test shapes (X, y): (30, 4) (30,)

Sklearn Implementation:
Train shapes (X, y): (120, 4) (120,)
Test shapes (X, y): (30, 4) (30,)


In [13]:
# Generate synthetic data
categories = ['red', 'green', 'blue']
data = np.random.choice(categories, size=100, replace=True)
data = data.reshape(-1, 1)  # Reshape for compatibility with encoders

# Initialize encoders
custom_one_hot_encoder = OneHotEncoder()
sklearn_one_hot_encoder = SklearnOneHotEncoder(sparse=False)

custom_label_encoder = LabelEncoder()
sklearn_label_encoder = SklearnLabelEncoder()

# Properly initialize OrdinalEncoder with an order
custom_ordinal_encoder = OrdinalEncoder(order=sorted(categories))
sklearn_ordinal_encoder = SklearnOrdinalEncoder(categories=[sorted(categories)])

def test_encoders(data):
    # One-Hot Encoding
    custom_one_hot_encoded = custom_one_hot_encoder.fit_transform(data)
    sklearn_one_hot_encoded = sklearn_one_hot_encoder.fit_transform(data)
    
    # Label Encoding - requires flattened data for Sklearn
    custom_label_encoded = custom_label_encoder.fit_transform(data.flatten())
    sklearn_label_encoded = sklearn_label_encoder.fit_transform(data.flatten())
    
    # Ordinal Encoding - custom implementation and sklearn
    custom_ordinal_encoded = custom_ordinal_encoder.fit_transform(data.flatten())
    sklearn_ordinal_encoded = sklearn_ordinal_encoder.fit_transform(data)

    # Compare results
    one_hot_difference = np.sum(custom_one_hot_encoded != sklearn_one_hot_encoded)
    total_elements = np.prod(sklearn_one_hot_encoded.shape)
    label_mse = mean_squared_error(custom_label_encoded, sklearn_label_encoded)
    ordinal_mse = mean_squared_error(custom_ordinal_encoded, sklearn_ordinal_encoded)

    print("One-Hot Encoding Total Different Elements:", one_hot_difference)
    print("One-Hot Encoding Percentage Difference:", one_hot_difference / total_elements)
    print("Label Encoding MSE:", label_mse)
    print("Ordinal Encoding MSE:", ordinal_mse)
    print("Custom One-Hot Shape:", custom_one_hot_encoded.shape)
    print("Sklearn One-Hot Shape:", sklearn_one_hot_encoded.shape)
    print("Custom One-Hot Example:", custom_one_hot_encoded[0])
    print("Sklearn One-Hot Example:", sklearn_one_hot_encoded[0])
    print("Custom Data Type:", custom_one_hot_encoded.dtype)
    print("Sklearn Data Type:", sklearn_one_hot_encoded.dtype)

# Run tests
test_encoders(data)


One-Hot Encoding Total Different Elements: 0
One-Hot Encoding Percentage Difference: 0.0
Label Encoding MSE: 0.0
Ordinal Encoding MSE: 0.0
Custom One-Hot Shape: (100, 3)
Sklearn One-Hot Shape: (100, 3)
Custom One-Hot Example: [1. 0. 0.]
Sklearn One-Hot Example: [1. 0. 0.]
Custom Data Type: float64
Sklearn Data Type: float64


In [14]:
# Generate synthetic data
X, y = make_classification(n_samples=100, n_features=20, n_informative=2, n_redundant=10, n_clusters_per_class=1, random_state=42)

def compare_selected_features(custom_selector, sklearn_selector, name):
    # Fit custom selector
    custom_selector.fit(X, y)
    custom_selected_indices = custom_selector.indices_

    # Fit sklearn selector
    sklearn_selector.fit(X, y)
    sklearn_support = sklearn_selector.get_support()
    sklearn_selected_indices = np.where(sklearn_support)[0]

    # Calculate overlap and differences
    intersection = np.intersect1d(custom_selected_indices, sklearn_selected_indices)
    difference = np.setdiff1d(custom_selected_indices, sklearn_selected_indices)

    # Report
    print(f"{name} - Custom selected indices: {custom_selected_indices}")
    print(f"{name} - Sklearn selected indices: {sklearn_selected_indices}")
    print(f"{name} - Intersection: {intersection}")
    print(f"{name} - Difference: {difference}")
    print(f"{name} - Overlap Percentage: {len(intersection) / len(sklearn_selected_indices) * 100:.2f}%\n")

# SelectKBest Comparison
custom_kbest = SelectKBest(score_func=f_classif, k=5)
sklearn_kbest = SklearnSelectKBest(score_func=f_classif, k=5)
compare_selected_features(custom_kbest, sklearn_kbest, "SelectKBest")

# SelectFromModel Comparison
custom_model_selector = SelectFromModel(model=RandomForestClassifier(random_state=42), threshold="mean")
sklearn_model_selector = SklearnSelectFromModel(estimator=RandomForestClassifier(random_state=42), threshold="mean")
compare_selected_features(custom_model_selector, sklearn_model_selector, "SelectFromModel")

# RFE Comparison
custom_rfe_selector = RFE(model=RandomForestClassifier(random_state=42), n_features_to_select=5)
sklearn_rfe_selector = SklearnRFE(estimator=RandomForestClassifier(random_state=42), n_features_to_select=5)
compare_selected_features(custom_rfe_selector, sklearn_rfe_selector, "RFE")


SelectKBest - Custom selected indices: [ 9  3 18  0 10]
SelectKBest - Sklearn selected indices: [ 0  3  9 10 18]
SelectKBest - Intersection: [ 0  3  9 10 18]
SelectKBest - Difference: []
SelectKBest - Overlap Percentage: 100.00%

SelectFromModel - Custom selected indices: [ 0  1  3  9 10 11 18 19]
SelectFromModel - Sklearn selected indices: [ 0  1  3  9 10 11 18 19]
SelectFromModel - Intersection: [ 0  1  3  9 10 11 18 19]
SelectFromModel - Difference: []
SelectFromModel - Overlap Percentage: 100.00%

RFE - Custom selected indices: [0, 3, 9, 10, 18]
RFE - Sklearn selected indices: [ 0  3  9 10 18]
RFE - Intersection: [ 0  3  9 10 18]
RFE - Difference: []
RFE - Overlap Percentage: 100.00%

